In [15]:
import requests
import pandas as pd
import json
from datetime import datetime
import mysql.connector

In [16]:
mydb = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = "admin",
    database = "sapo_data",
    charset = "utf8mb4"
) 

In [17]:
cookies = 'your cookie'

In [18]:
url = "https://yoururl/admin/stock_transfers.json?page=1&limit=20"
payload = {}
headers = {
  'Cookie': cookies,
}

response = requests.request("GET", url, headers=headers, data=payload)

In [19]:
def get_product_name_sku_unit(variant_id: int) -> (str, str, str):
    product_url = f"https://yoururl/admin/variants/search.json?ids={variant_id}&status=active%2Cinactive%2Cdeleted&page=1&limit=200"
    product_payload = {}
    product_headers = {
      'Cookie': cookies,
    }
    
    product_response = requests.request("GET", product_url, headers=product_headers, data=product_payload)
    product_data = json.loads(product_response.text)
    product_name = product_data["variants"][0]["name"]
    product_sku = product_data["variants"][0]["sku"]
    product_unit = product_data["variants"][0]["unit"]
    return product_name, product_sku, product_unit
    

In [20]:
data: dict = json.loads(response.text)

In [21]:
created_ons = []
product_names = []
product_skus = []
source_locations = []
destination_locations = []
lot_numbers = []
manufacture_dates = []
expiration_dates = []
type_sts = []
quantities = []
product_units = []
codes = []
notes = []

In [22]:
def insert_stock_data(created_on,product_name,product_sku,source_location,destination_location,lot_number,manufacture_date,expiration_date,type_st,quantity,product_unit,code,note):
    mycursor = mydb.cursor()
    sql= "insert ignore into `stock`(created_on,product_name,product_sku,source_location,destination_location,lot_number,manufacture_date,expiration_date,type_st,quantity,product_unit,code,note) values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
    val = (created_on,product_name,product_sku,source_location,destination_location,lot_number,manufacture_date,expiration_date,type_st,quantity,product_unit,code,note)
    try:
        mycursor.execute(sql, val)
        
        mydb.commit()
    except:
        raise  

In [23]:
locations = {
    598184: "CN HCM",
    598800: "Xưởng",
    719281: "KIOS",
    727076: "CN Bình Thuận"
}

In [24]:
for stock_transfers in data['stock_transfers']:
    created_on = stock_transfers['created_on']
    created_on = datetime.strptime(created_on,"%Y-%m-%dT%H:%M:%SZ").strftime("%d/%m/%Y")
    code = stock_transfers['code']
    source_location = locations[stock_transfers["source_location_id"]]
    destination_location = locations[stock_transfers["destination_location_id"]]
    if stock_transfers["destination_location_id"]==727076:
        continue
    for line_item in stock_transfers['line_items']:
        lots_dates = line_item["lots_dates"]
        variant_id = line_item["variant_id"]
        product_name, sku, unit = get_product_name_sku_unit(variant_id)
        for lots_date in lots_dates:
            lots_number = lots_date["lots_number"]
            manufacture_date = lots_date["manufacture_date"]
            manufacture_date = datetime.strptime(manufacture_date,"%Y-%m-%dT%H:%M:%SZ").strftime("%d/%m/%Y")
            expiration_date = lots_date["expiration_date"]
            expiration_date = datetime.strptime(expiration_date,"%Y-%m-%dT%H:%M:%SZ").strftime("%d/%m/%Y")
            type_st = "CHUYỂN"
            quantity = lots_date["quantity"]
            #save data
            created_ons.append(created_on)
            product_names.append(product_name)
            product_skus.append(sku)
            source_locations.append(source_location) 
            destination_locations.append(destination_location)
            lot_numbers.append(lots_number)
            manufacture_dates.append(manufacture_date)
            expiration_dates.append(expiration_date)
            type_sts.append(type_st) 
            quantities.append(quantity) 
            product_units.append(unit) 
            codes.append(code)
            notes.append("")
            insert_stock_data(created_on,product_name,sku,source_location,destination_location,lots_number,manufacture_date,expiration_date,type_st,quantity,unit,code, "")

In [25]:
data = {
    'Created On': created_ons,
    'Product Name': product_names,
    'Product SKU': product_skus,
    'Source Location': source_locations,
    'Destination Location': destination_locations,
    'Lot Number': lot_numbers,
    'Manufacture Date': manufacture_dates,
    'Expiration Date': expiration_dates,
    'Type': type_sts,
    'Quantity': quantities,
    'Product Unit': product_units,
    'Code': codes,
    'Notes': notes
}
df = pd.DataFrame(data)

In [26]:
df.to_csv('stock.csv', index=False)

In [27]:
mydb.close()